In [8]:
from pathlib import Path
import pandas as pd
import json

In [9]:
data_path = Path('data/')

In [10]:
summoner_folders = []
for summoner_folder in data_path.iterdir():
    if str(summoner_folder.name).startswith('.'):
        continue
    print(summoner_folder)
    summoner_folders.append(summoner_folder)

data/3Ejr6Hrg4kUbhdt8Ldj8idgYyIMxeqxGUCcW1bRK35T2jsy5OF0ad-g7_j7fgGQBI8PSXrWvfi1jOg
data/VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw


In [38]:
import flatdict

def get_all_games_per_user(user_folder):
    participant_one_hot = ['championId', 'individualPosition','role', 'summoner1Id', "summoner2Id"]

    participant_drop = ['perks','challenges','championName', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'lane', 'puuid', 'riotIdName', 'riotIdTagline', 'summonerId', 'summonerName', 'teamEarlySurrendered', 'teamId', 'teamPosition']
    
    
    all_games_df = pd.DataFrame()
    for game_file in user_folder.iterdir():
        user_id = user_folder.name.split('.')[0]
        with open(game_file, 'r') as f:
            game_json = json.load(f)

            game_info_dict = {
                'user_id': user_id
            }
            metadata = game_json['metadata']
            game_info_dict = {**game_info_dict, **metadata}
            game_info_dict['num_participants'] = len(metadata['participants']) 
            game_info = game_json['info']

            
            team_id_won = -1
            for team in game_info['teams']:
                if team['win'] == True:
                    team_id_won = team['teamId']
            game_info_dict['team_won'] = team_id_won
            
            user_id_index = metadata['participants'].index(user_id)
            for i, participant_info in enumerate(game_info['participants']):
                for col in participant_drop:
                    del participant_info[col]
                participant_info_flattened_orig = flatdict.FlatDict(participant_info, delimiter='.')
                participant_info_flattened = {f'participant{i}_{k}':v for (k,v) in participant_info_flattened_orig.items()}
                
                game_info_dict = {**game_info_dict, **participant_info_flattened}
                if user_id_index == i:
                    my_info = {f'participant_me_{k}':v for (k,v) in participant_info_flattened_orig.items()}
                    game_info_dict = {**game_info_dict, **my_info}
                
            game_info_dict['primary_part_index'] = user_id_index
            del game_info['participants']
            game_info_dict = {**game_info_dict, **game_info}

            game_info_df = pd.DataFrame([game_info_dict])

            all_games_df = pd.concat([all_games_df, game_info_df])
    return all_games_df.reset_index(drop=True)

In [39]:
all_users_df = pd.DataFrame()

# for user_path in summoner_folders[:1]:
#     print(user_path)
#     all_users_df = pd.concat([all_users_df, get_all_games_per_user(user_path)])
user_path = Path('data/VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw')
print(user_path)
all_users_df = pd.concat([all_users_df, get_all_games_per_user(user_path)])
all_users_df

data/VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw


,user_id,dataVersion,matchId,participants,num_participants,team_won,participant0_assists,participant0_baronKills,participant0_bountyLevel,participant0_champExperience,...,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,platformId,queueId,teams,tournamentCode
0,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4289108259,[V-yGH9BqElmFYCHmjCtQT8VZbe1LZQ3Z5CNfqk47OV5Nx...,10,200,7,0,0,10900,...,CLASSIC,teambuilder-match-4289108259,1650991247325,MATCHED_GAME,12.7.433.4138,11,NA1,420,"[{'bans': [{'championId': 164, 'pickTurn': 1},...",
1,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4289841360,[xWXDqDrLxGFzTknwyXiK3XcBw6WNnY6FCWk1HV24IDNLX...,10,200,5,0,0,11408,...,CLASSIC,teambuilder-match-4289841360,1651027780889,MATCHED_GAME,12.7.433.4138,11,NA1,420,"[{'bans': [{'championId': 246, 'pickTurn': 1},...",
2,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4303183995,[aaryaHyLHTYYszidooyeOnFujtYsv8zdBkgdz8nY5hQPr...,10,200,9,0,0,11691,...,CLASSIC,teambuilder-match-4303183995,1652210568022,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 203, 'pickTurn': 1},...",
3,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4289603022,[zE_XKZyQNTFcKd0sWfOqnieoxiqU9i262Kbhws6qw8cjs...,10,100,12,0,0,12196,...,CLASSIC,teambuilder-match-4289603022,1651017021612,MATCHED_GAME,12.7.433.4138,11,NA1,420,"[{'bans': [{'championId': 157, 'pickTurn': 1},...",
4,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4290454390,[7uSClZHCUGeyM9mf1oznT06HgL1diJ-AzKf8_uLqpNlRu...,10,200,5,0,0,10942,...,CLASSIC,teambuilder-match-4290454390,1651076791014,MATCHED_GAME,12.8.436.3750,11,NA1,420,"[{'bans': [{'championId': 497, 'pickTurn': 1},...",
5,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4303154240,[0ZMWUHRsdqa1H8D0TUSTvGQeoypU6tSwayxyx3E3rvpNp...,10,200,3,0,0,12446,...,CLASSIC,teambuilder-match-4303154240,1652208335272,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 234, 'pickTurn': 1},...",
6,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4289161615,[Pftgib0GaonAminW_vk2oBgwWHO0dOppwNnNNBcu44_Wu...,10,100,7,0,0,14476,...,CLASSIC,teambuilder-match-4289161615,1650993961683,MATCHED_GAME,12.7.433.4138,11,NA1,420,"[{'bans': [{'championId': 104, 'pickTurn': 1},...",
7,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4290458864,[OOzlPC2MSvtb_En8N7rgviDgfqSK1Z2tirSswKd2euRNK...,10,200,0,0,0,12037,...,CLASSIC,teambuilder-match-4290458864,1651080776872,MATCHED_GAME,12.8.436.3750,11,NA1,420,"[{'bans': [{'championId': 246, 'pickTurn': 1},...",
8,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4291330895,[B7TLNkIFiMKtsYgrRGg6eFnYG_INNXGBE4WYlGA774GFT...,10,100,20,0,0,19464,...,CLASSIC,teambuilder-match-4291330895,1651165205384,MATCHED_GAME,12.8.436.3750,11,NA1,420,"[{'bans': [{'championId': 67, 'pickTurn': 1}, ...",
9,VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AV...,2,NA1_4289847932,[s4zELEwgdK-iiW3XPdx2bOUY64mf3Ucmk5S0Ny5FVkYCN...,10,100,7,0,2,14522,...,CLASSIC,teambuilder-match-4289847932,1651029879394,MATCHED_GAME,12.7.433.4138,11,NA1,420,"[{'bans': [{'championId': 38, 'pickTurn': 1}, ...",


In [40]:
all_users_df.loc[0, 'primary_part_index']

9

In [47]:
ml_drop_cols = ['matchId', 'dataVersion', 'participants', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'platformId', 'queueId', 'tournamentCode', 'team_won', 'num_participants']           

first_user_filter_df = all_users_df.loc[:, all_users_df.columns.str.startswith('participant_me_') | all_users_df.columns.isin(['user_id'])]
per_user_stats_df = first_user_filter_df.groupby('user_id').mean()
per_user_stats_df

,participant_me_assists,participant_me_baronKills,participant_me_bountyLevel,participant_me_champExperience,participant_me_champLevel,participant_me_championId,participant_me_championTransform,participant_me_consumablesPurchased,participant_me_damageDealtToBuildings,participant_me_damageDealtToObjectives,...,participant_me_trueDamageTaken,participant_me_turretKills,participant_me_turretTakedowns,participant_me_turretsLost,participant_me_unrealKills,participant_me_visionScore,participant_me_visionWardsBoughtInGame,participant_me_wardsKilled,participant_me_wardsPlaced,participant_me_win
user_id,,,,,,,,,,,,,,,,,,,,,
VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw,13.45,0.0,0.25,9622.25,12.15,174.3,0.0,13.85,542.05,1836.85,...,731.85,0.25,2.05,5.7,0.0,61.7,10.15,7.1,30.85,0.5


In [47]:
ml_drop_cols = ['matchId', 'dataVersion', 'participants', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'platformId', 'queueId', 'tournamentCode']           